In [10]:
import numpy as np

ID_hxn_connect = .824 #in
g_imp = 32.2
g_met = 9.8

Thotin = 40 #C
Tcoldin = 25 #C
Vhotin = 25 #lpm
Vhotin_f = 0.01471444 #ft^3/s
rho_water_m = .992 #kg/l
MhotinSI = Vhotin*rho_water_m/60 #kg/s
cp_water_m = 4.18 #kj/kg/C
heat_req = 11 #kw
deltaThot = heat_req/(MhotinSI*cp_water_m) #C
Thotout = Thotin -  deltaThot #C

rho_water = 62.4
mu_water = .926

def reynolds_f(density,D,vel,mu): #ASSUMPTION: D in ft and vel in ft^3/s
    return density*D*vel/mu

def Tlogmean(hotin,coldin,hotout,coldout):
    return ((hotout-coldin)-(hotin+coldout))/np.log((hotout-coldin)/(hotin-coldout))

class heat_ex:
    def __init__(self,SA,PL,k24,k31,nop):
        self.SA = SA
        self.PL = PL
        self.k24 = k24
        self.k31 = k31
        self.nop = nop
        self.pipe_D = ID_hxn_connect*.0254 #m
        self.pipe_A = self.pipe_D**2/4*np.pi #m^2
    def set_flow(self,Vdot): #ASSUMPTION: Vdot is in lpm
        self.massdot = Vdot*rho_water_m/60 #kg/s
        self.Vdot = Vdot*0.0000166667 #m^3/s
        print(self.pipe_A,self.Vdot)
        self.velocity = self.Vdot/self.pipe_A
    def presdrop_3_1(self):
        self.presdrop = self.k31*self.velocity**2/(2*g_met) #Pascals!!
    def calc_re(self):
        cur_vel = min(Vhotin_f,self.Vdot)
        self.re = reynolds_f(rho_water,self.pipe_D,cur_vel,mu_water) #This is the reynolds for U calculations only!!!
        #AKA, this will be the reynolds for the lowest flowrate stream
    def calc_U(self):
        self.U = self.re/self.nop+1800 #W/m^2/K
    def calc_Tcoldout(self):
        deltaTcold = heat_req/(self.massdot*cp_water_m)
        self.Tcoldout = Tcoldin-deltaTcold
    def double_check(self):
        Q = self.U*self.SA*Tlogmean(Thotin,Tcoldin,Thotout,self.Tcoldout)
        print(Q,heat_req*1000)
        
hxn10 = heat_ex(.12,7.5,209,102,10)
hxn10.set_flow(40)
hxn10.presdrop_3_1()
print(hxn10.presdrop)
hxn10.calc_re()
hxn10.calc_U()
hxn10.calc_Tcoldout()
hxn10.double_check()

0.0003440422173277026 0.000666668
19.540687681577463
14365.143146683835 11000
